# DevMuT Artifact

Artifact of the paper "DevMuT: Testing Deep Learning Framework via Developer
Expertise-Based Mutation" from ASE 2024.

To access ChameleonCloud resources, you may need the account to log in to ChameleonCloud. You also need to have a project to allocate resources (e.g., node).

In [1]:
from chi import context

context.version = "1.0"

context.choose_site(default="CHI@TACC")
context.choose_project()

### Check available hardware

In [2]:
from chi import hardware

node_type = "compute_cascadelake_r"
available_nodes = hardware.get_nodes(node_type=node_type, filter_reserved=True)
if available_nodes:
    print(f"There currently are {len(available_nodes)} {node_type} nodes ready to use")
else:
    print(f"All {node_type} nodes are in use! You could use next_free_timeslot to see how long you need to wait, or use the calendar.")

There currently are 8 compute_cascadelake_r nodes ready to use


### Reserve node

In [3]:
from chi import lease
from datetime import timedelta
import os

my_lease = lease.Lease(f"{os.getenv('USER')}-power-management", duration=timedelta(hours=3))
my_lease.add_node_reservation(nodes=[available_nodes[0]]) # or you could use node_type=node_type
my_lease.add_fip_reservation(1) # include a floating ip
my_lease.submit(idempotent=True)

Waiting for lease to start... This can take up to 60 seconds


Lease radhofanazizi_gmail_com-power-management has reached status active


### Create a server on the node

In [4]:
from chi import server

my_server = server.Server(
    f"{os.getenv('USER')}-power-management",
    reservation_id=my_lease.node_reservations[0]["id"],
    image_name="CC-Ubuntu22.04", # or use image_name
)
my_server.submit(idempotent=True)

Waiting for server radhofanazizi_gmail_com-power-management's status to become ACTIVE. This typically takes 10 minutes, but can take up to 20 minutes.


Server has moved to status ACTIVE


Attribute,radhofanazizi_gmail_com-power-management
Id,5c96c890-3907-4cf4-a5b2-884a2f390edc
Status,ACTIVE
Image Name,CC-Ubuntu22.04
Flavor Name,baremetal
Addresses,sharednet1: IP: 10.52.2.79 (v4) Type: fixed MAC: bc:97:e1:c4:b2:f0
Network Name,sharednet1
Created At,2025-03-11T08:33:44Z
Keypair,trovi-c805e44
Reservation Id,aeb1eb5f-a066-4653-8bad-5262e395dd07
Host Id,b281b13a05d4a4d342f673906de4005142c2819a049809e34ac97306


### Configure networking on the node

In [5]:
fip = my_lease.get_reserved_floating_ips()[0]
my_server.associate_floating_ip(fip)
my_server.check_connectivity(host=fip)

Checking connectivity to 129.114.109.233 port 22.


Connection successful


## Run DevMuT

Now, we can finally run DevMuT. First we need to clone the github repo first and then run the reprduce.sh script which contain the instructions from README.md
packaged into a bash file

In [80]:
my_server.execute("rm -rf DevMuT")

<Result cmd='rm -rf DevMuT' exited=0>

In [81]:
my_server.execute("git clone https://github.com/radhofan/DevMuT.git")

Cloning into 'DevMuT'...
Updating files: 100% (3435/3435), done.


<Result cmd='git clone https://github.com/radhofan/DevMuT.git' exited=0>

In [82]:
my_server.execute("chmod +x DevMuT/reproduce.sh")
my_server.execute("bash DevMuT/reproduce.sh")

ERROR: File or directory already exists: '/home/cc/miniconda'
If you want to update an existing installation, use the -u option.


Channels:
 - defaults
Platform: linux-64
Solving environment: ...working... done

## Package Plan ##

  environment location: /home/cc/miniconda/envs/DevMuT

  added / updated specs:
    - python=3.9


The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main 
  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu 
  ca-certificates    pkgs/main/linux-64::ca-certificates-2025.2.25-h06a4308_0 
  ld_impl_linux-64   pkgs/main/linux-64::ld_impl_linux-64-2.40-h12ee557_0 
  libffi             pkgs/main/linux-64::libffi-3.4.4-h6a678d5_1 
  libgcc-ng          pkgs/main/linux-64::libgcc-ng-11.2.0-h1234567_1 
  libgomp            pkgs/main/linux-64::libgomp-11.2.0-h1234567_1 
  libstdcxx-ng       pkgs/main/linux-64::libstdcxx-ng-11.2.0-h1234567_1 
  ncurses            pkgs/main/linux-64::ncurses-6.4-h6a678d5_0 
  openssl            pkgs/main/linux-64::openssl-3.0.16-h5eee18b_0 
  pip                pkgs/main/linux-64::pip-25.0-py39h0


CondaError: Run 'conda init' before 'conda activate'



Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... failed



PackagesNotFoundError: The following packages are not available from current channels:

  - absl-py==0.15.0=pypi_0

Current channels:

  - https://conda.anaconda.org/conda-forge
  - https://repo.anaconda.com/pkgs/main
  - https://repo.anaconda.com/pkgs/r

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




Channels:
 - pytorch
 - nvidia
 - defaults
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



Traceback (most recent call last):
  File "/home/cc/DevMuT/code/DevMuT/mutation_test.py", line 8, in <module>
    import torch
  File "/home/cc/miniconda/lib/python3.12/site-packages/torch/__init__.py", line 367, in <module>
    from torch._C import *  # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^^
ImportError: /home/cc/miniconda/lib/python3.12/site-packages/torch/lib/libtorch_cpu.so: undefined symbol: iJIT_NotifyEvent


UnexpectedExit: Encountered a bad command exit code!

Command: 'bash DevMuT/reproduce.sh'

Exit code: 1

Stdout: already printed

Stderr: already printed

